In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing Libraries

In [ ]:
#!pip install --upgrade scikit-learn
#pip install imblearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold,cross_val_score
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from catboost import CatBoostClassifier

from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

## Load Data

In [ ]:
train_df = pd.read_csv('/kaggle/input/dont-overfit-ii/train.csv')
test_df = pd.read_csv('/kaggle/input/dont-overfit-ii/test.csv')

## Data exploration

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
print(train_df.shape,test_df.shape)

In [ ]:
train_df.info()

###### id column is the only int data type

In [ ]:
train_df.describe().T

In [ ]:
train_df.isnull().sum().sum()

In [ ]:
train_df['target'].value_counts()

In [ ]:
sns.countplot(train_df['target'])

In [ ]:
zeros = train_df['target'].value_counts()[0]
ones = train_df['target'].value_counts()[1]
num_records = train_df.shape[0]
print('classes percentage%: ',f'zeros: {zeros / num_records * 100}%')
print(f'ones {ones / num_records * 100}%')

In [ ]:
plt.hist(train_df[train_df.columns[2:]].mean())
plt.title('Distribution of features mean')
plt.ylabel('frequency')

In [ ]:
corr = train_df.corr().abs().unstack().sort_values(kind="quicksort").reset_index()
corr = corr[corr['level_0'] != corr['level_1']]

corr.tail(10)

##### there is no high correlation between features to be dropped

In [ ]:
for i in range(300):
    char = str(i)
    train_df[char].hist()
    plt.show()
    

**features are almost normally distribured,so no need to transform**

## Splitting data

In [ ]:
x_train = train_df.drop(['id', 'target'], axis=1)
y_train = train_df['target']
t=test_df['id']
x_test = test_df.drop(['id'], axis=1)


## Data is imbalanced

In [ ]:
smt = SMOTE(random_state=0)
balanced_x_train, balanced_y_train = smt.fit_resample(x_train,y_train)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(balanced_x_train,balanced_y_train, train_size=0.8, test_size=0.2,random_state=0)

In [ ]:
models=[LogisticRegression(),CatBoostClassifier(),RandomForestClassifier(),AdaBoostClassifier(),BaggingClassifier(),DecisionTreeClassifier(),SGDClassifier()]
model_names=['LogisticRegression','CatBoostClassifier','RandomForestClassifier','AdaBoostClassifier','BaggingClassifier','DecisionTreeClassifier','SGDClassifier']
roc=[]
d={}


cv = KFold(n_splits = 5,random_state=0, shuffle=False)

for model in range (len(models)):
    clf=models[model]
    scores = cross_val_score(clf,x_train,y_train,scoring='roc_auc',cv=cv,verbose=1)
    roc.append(np.mean(scores))
    
    #clf.fit(x_train,y_train)
    #print("model_name : ",model_names[model])
    #print(clf.get_params())
    #test_pred=clf.predict(x_valid)
    #print(test_pred)
    
    
d={'Modelling Algo':model_names,'Roc':roc}   
d

In [ ]:
roc_frame=pd.DataFrame(d)
roc_frame

In [ ]:
sns.factorplot(y='Modelling Algo',x='Roc',data=roc_frame,kind='bar',size=5,aspect=2)

In [ ]:
def scoring_roc_auc(y, y_pred):
    try:
        return roc_auc_score(y, y_pred)
    except:
        return 0.5
from sklearn.metrics import make_scorer
roc_auc = make_scorer(scoring_roc_auc)

In [ ]:
clf=DecisionTreeClassifier()
clf.fit(x_train, y_train)

In [ ]:
pred=clf.predict(x_valid)
score=roc_auc_score(y_valid,pred)
score

In [ ]:
prediction= clf.predict(x_test)

## Submission file

In [ ]:
predictions = pd.DataFrame({'id':t,
                       'target': prediction})

In [ ]:
predictions.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")